In [2]:
import speech_recognition as sr
import pandas as pd

In [2]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project sunny-lightning-392907

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=5Nr3Ji4w9bNpzGDRpwUSk7eAWsSsky&access_type=offline&code_challenge=H7OKydl6xCAoJy0R_iL3ta-tys4ts7GMMNzEwpsBRUM&code_challenge_method=S256


Credentials saved to file: [C:\Users\vadim\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "hack-team-fdw" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You m

In [3]:
context = """
You are an assistant who helps users understand more about our application. You need to answer in one or two sentences.
Users will ask you questions and you will answer them given the below information. Don't answer in more than three sentences:

About the application:
Our application is an outlook plugin which helps users to analyze and write their emails more efficiently. it has the following functions:
1. It can show a sentiment of email, i.e. negative or positive. This will help users to avoid sending negative emails and try to write more positive emails
2. It can provide a language tone of email, this should help users to avoid impolite words and avoid using inappropriate language
3. It can summarize long emails in a few sentences. This can save time for users and avoid reading long email chains
4. It can rewrite an email in more polite and positive language.
5. It can translate to other languages

User question:
"""

In [4]:
context = """
You are the interactive Banking Unicorn assistant at our exhibition stand fo Your role is to engage with attendees, answer questions about the app, and provide information about our team and company..
About the application:
Our application is an outlook plugin which helps users to analyze and write their emails more efficiently. it has the following functions:
1. It can show a sentiment of email, i.e. negative or positive. This will help users to avoid sending negative emails and try to write more positive emails
2. It can provide a language tone of email, this should help users to avoid impolite words and avoid using inappropriate language
3. It can summarize long emails in a few sentences. This can save time for users and avoid reading long email chains
4. It can rewrite an email in more polite and positive language.
5. It can translate to other languages

MailboxIQ is a cutting-edge AI-powered application designed to revolutionize email communication. It's your intelligent email assistant that streamlines your inbox and makes managing emails more efficient and insightful. The app offers features such as email summaries, sentiment analysis, priority sorting, customizable filters, and language translation. It seamlessly integrates with your existing email client.

If there are questions about the app or team that you cannot answer, please direct attendees to the person at the exhibition stand for assistance. For general queries not related to the app or team, such as asking about the weather in a specific city, you can use your ability to access the internet to provide relevant information.

You need to answer in one or two sentences.
Users will ask you questions and you will answer them given the below information. Don't answer in more than three sentences.

User question:
"""

In [5]:
from vertexai.preview.language_models import TextGenerationModel
import vertexai

def send_user_input(project_id, location, question, temperature: float = .1):
    vertexai.init(project=project_id, location=location)
    
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 128,
        "top_p": .8,
        "top_k": 40,
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    request_text = context + question 
    #print(request_text)
    response = model.predict(request_text, **parameters)
    #print(f"Response from Model: {response.text}")
    return response.text
   


In [6]:
df = pd.read_excel("Answers/HackathonQuestionsExcel.xlsx",header=0,converters={'id':str})


In [7]:
questions = list(df.iloc[:, 1].values)
questions

["What's the name of the Banking Unicorn?",
 'What is your name?',
 'How does the Banking Unicorn work?',
 'How do you work?',
 'What is inside you?',
 'Can the Banking Unicorn learn and improve its responses over time?',
 'Can you be trained?',
 'Is the Banking Unicorn based on a specific AI technology or framework?',
 "What's the purpose of having an Banking Unicorn at your exhibition stand?",
 'Can the Banking Unicorn interact with other AI technologies or devices?',
 'Can the one customize you?',
 'Is the Banking Unicorn customizable in any way?',
 'What are your limitations?',
 "What are the limitations of the Banking Unicorn's capabilities?",
 'Can I take a picture with you?',
 'Can I take a picture with the Banking Unicorn?',
 'How are you today?',
 'How are you doing?',
 'How are you?',
 'What is MailboxIQ and what does it do?',
 'What inspired you to develop the MailboxIQ App?',
 'How can the MailboxIQ App help individuals or businesses?',
 'Is the MailboxIQ App available for 

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('stsb-roberta-large')

embeddings = model.encode(questions)

In [9]:
def get_score(model, test_sentence, questions):
    test_emb = model.encode(test_sentence)
    test_emb = test_emb.reshape(1, -1)
    max_cs = 0
    max_idx = -1
    for i, emb in enumerate(embeddings):
        cs = cosine_similarity(test_emb, emb.reshape(1, -1))[0][0]
        if cs > max_cs:
            max_cs = cs
            max_idx = i    
    return (max_cs, max_idx)
print(get_score(model, "What's MailboxIQ?", questions))

(0.94933844, 19)


In [10]:
project_id="sunny-lightning-392907"
location="us-central1"
output_voice_file="response.mp3"
output_voice_file_changed = "response_changed.mp3"
idle_video_file = "assets/idle9s.mp4"
#special videos:
special_videos = [ ("assets/hello.mp4", "assets/hello.mp3"), 
                  ("assets/struggling.mp4", "assets/struggling.mp3"), 
                  ("assets/curious.mp4", "assets/curious.mp3"), 
                  ("assets/space_button.mp4", "assets/space_button.mp3"), 
                  ("assets/you_can_ask_me.mp4", "assets/you_can_ask_me.mp3")
                 ]
hello_video_file = "assets/hello.mp4"
hello_audio_file = "assets/hello.mp3"
#other videos
listen_video_file = "assets/listen_long.mp4"
speaking_video_file = "assets/speaking.mp4"

In [14]:
#Vadim's Laptop
screen_size = [1920,1080]
video_y_pos = 150
micro_pos = [1200,710]
space_key_text_pos = [550,800]
speak_text_pos = [800,800]
ask_me_text_pos = [1370, 200]
text_question_pos = [550,750]
text_question_color = (255,255,0)
text_response_pos = [800,800]
text_response_color = (255,0,0)
copyright_pos = [550,870]
copyright_color = (0,0,0)
copyright_font_size=12
online_status_pos = [1560,870]
online_status=True

text_reponse_shift_speed = 7

sample_questions_img_pos = [1360, 180]
video_scale = 0.8

# predefined questions, similarity threshold
similarity_threshold = 0.5

# parameters
voice_shift = 7
adjust_for_ambient_noise_duration = 0.4
log_file = 'conversation.log'
mic_device_index = 1 # sr.Microphone.list_microphone_names()

#default answer default_answer_idx
default_answer_idx = df.loc[df.questions == "Default"].index[0]

In [15]:
import pygame
from pygame.locals import *
import cv2
import numpy as np
import sys
from gtts import gTTS
from playsound import playsound
import cv2
import librosa
import soundfile as sf
import IPython
from IPython.display import Audio
from threading import Thread, Event
from enum import Enum
from pygame import mixer
import os
import speech_recognition as sr
import shutil
from random import random, randint

def log(*msg):
    print(*msg)
    with open(log_file, 'a') as f:
        for m in msg:
            f.write(str(m))
            f.write(' ')
        f.write('\n')
    
class VoiceRecordingThread(Thread):
    def run(self):
        self.response = ""
        self.video_response = ""
        self.audio_response = ""
        self.question = ""
        self.thinking = False
        r = sr.Recognizer()
        self.recognizer = r
        with sr.Microphone(device_index=mic_device_index) as source:
            r.adjust_for_ambient_noise(source, duration=adjust_for_ambient_noise_duration)
            log("speak")
            audio = r.listen(source)
        user_input = r.recognize_google(audio)
        self.question = user_input
        log("User input:", user_input)
        score, idx = get_score(model, user_input, questions)
        log("scores:", score, idx)
        if score > similarity_threshold:
            log("use predefined question: ", questions[idx])
            file_id = df.iloc[idx, 0]
            shutil.copyfile("Answers/"+file_id+"_response_unicorn.mp3", output_voice_file_changed)
            self.video_response = "Answers/"+file_id+"_response_unicorn.mp4"
            self.audio_response = "Answers/"+file_id+"_response_unicorn.mp3"
            
            self.response = df.iloc[idx, 2]
        elif online_status: # online
            log("online, sending request")
            self.thinking = True
            self.response = send_user_input(project_id, location, user_input)
            log("Model response:", self.response)
            voice_prep(self.response)
            self.video_response = speaking_video_file
            self.audio_response = output_voice_file_changed
            self.thinking = False
        else:
            log("offline, default response")
            file_id = df.iloc[default_answer_idx, 0]
            shutil.copyfile("Answers/"+file_id+"_response_unicorn.mp3", output_voice_file_changed)
            self.video_response = "Answers/"+file_id+"_response_unicorn.mp4"
            self.audio_response = "Answers/"+file_id+"_response_unicorn.mp3"
            self.response = df.iloc[default_answer_idx, 2]
     
    
class PlaySoundThread(Thread):
    def __init__(self, stop_event, voice_file=output_voice_file_changed):
        Thread.__init__(self)
        self.stop_event = stop_event
        self.voice_file = voice_file
        
    def run(self):
        mixer.init()
        mixer.music.load(self.voice_file)
        mixer.music.play()
        while(mixer.music.get_busy() and not self.stop_event.is_set()):
            pass
        mixer.music.unload()
        log("audio end")

        
def voice_prep(response):
    language = 'en'

    speech = gTTS(text=response, lang=language)

    speech.save(output_voice_file)
    save_shifted_pitch(output_voice_file, output_voice_file_changed, voice_shift)
    #save_shifted_pitch("response", 7, )
    #os.system("start response_shifted.wav")
    #os.system("start " + output_voice_file)
    #playsound(output_voice_file)

def printText(frame, text, pos = 50):
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (pos, 1000)
    fontScale = 2
    color = (0, 255, 255)
    thickness = 4
    frame = cv2.putText(frame, text, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    return frame
    
class States(Enum):
    IDLE = 1
    LISTENING = 2
    SPEAKING = 3

def unicornVideo():
    global online_status
    pygame.init()

    # prepare assets
    micro_img_white = pygame.image.load('assets/micro_white.png')
    micro_img_black = pygame.image.load('assets/micro.png')
    micro_img = micro_img_black

    big_font = pygame.font.SysFont('comicsansms', 40)
    space_key_text = big_font.render('Press and hold Space key to speak', True, (0,0,0))
    speak_text = big_font.render('please speak', True, (0,0,0))
    thinking_text = big_font.render('thinking..', True, (0,0,0))

    small_font = pygame.font.SysFont('comicsansms', 24)
    ask_me_text = small_font.render('What can you ask me?', True, (255,255,255))
    sample_questions_img = pygame.image.load('assets/sample_questions.png')
    
    copyright_font = pygame.font.SysFont('comicsansms', copyright_font_size)
    copyright_text = copyright_font.render('Created by: Vadim Pidonenko, Radik Khamidullin', True, copyright_color)
    
    very_small_font = pygame.font.SysFont('comicsansms', 12)
    online_text_on = very_small_font.render('On', True, (255,255,255))
    online_text_off = very_small_font.render('Off', True, (255,255,255))
    
    # current video

    cap = cv2.VideoCapture(idle_video_file)

    #pygame.display.set_caption("OpenCV camera stream on Pygame")
    screen = pygame.display.set_mode(screen_size)
    shape = None
    
    #
    
    speaking_thread = None
    listening_thread = None
    state = States.IDLE
    text_response = None
    text_reponse_shift = 0
    text_question = None   
    hello_played = False
    stop_event = None
    try:
        while(cap.isOpened()):
            if state == States.LISTENING:
                if not listening_thread.is_alive():
                    #print("thread not alive")
                    text_response = listening_thread.response
                    text_question = listening_thread.question
                    text_reponse_shift = 0
                    audio_response = listening_thread.audio_response
                    video_response = listening_thread.video_response
                    log(audio_response)
                    log(video_response)
                    listening_thread = None
                    state = States.SPEAKING
                    stop_event = Event()
                    if audio_response:
                        speaking_thread = PlaySoundThread(stop_event, audio_response)
                    else:
                        speaking_thread = PlaySoundThread(stop_event)
                        
                    if text_response:
                        speaking_thread.start()
                        cap.release()
                        if video_response and os.path.isfile(video_response): # check if video response file exists
                            cap = cv2.VideoCapture(video_response)
                        else:
                            cap = cv2.VideoCapture(speaking_video_file)
            elif state == States.SPEAKING:
                if not speaking_thread.is_alive():
                    speaking_thread = None
                    state = States.IDLE
                    cap.release()
                    cap = cv2.VideoCapture(idle_video_file)
                    
            ret, frame = cap.read()
            if ret:
                screen.fill([0,0,0])
                shape = (frame.shape[0]*video_scale, frame.shape[1]*video_scale)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = np.rot90(frame)
                frame = pygame.surfarray.make_surface(frame)
                if video_scale != 1:
                    frame = pygame.transform.scale_by(frame, video_scale)
                    
                if screen_size[0] > shape[0]:
                    screen.blit(frame, ( (screen_size[0] - shape[0])/2, video_y_pos))
                else:
                    screen.blit(frame, (0,0))

                #display UI
                screen.blit(micro_img, micro_pos)
                screen.blit(sample_questions_img, sample_questions_img_pos)

                # display text
                if state == States.IDLE:
                    screen.blit(space_key_text, space_key_text_pos)
                elif state == States.LISTENING:
                    if listening_thread and listening_thread.thinking:
                        screen.blit(thinking_text, speak_text_pos)
                    else:
                        screen.blit(speak_text, speak_text_pos)
                elif state == States.SPEAKING:
                    if text_question:
                        screen.blit(small_font.render("Your question:" +text_question, True, text_question_color, (0,0,0)), text_question_pos)
                    if text_response:
                        text_reponse_shift += text_reponse_shift_speed
                        screen.blit(small_font.render(text_response, True, text_response_color, (0,0,0)),
                                    [text_response_pos[0]-text_reponse_shift, text_response_pos[1]])

                if online_status:
                    screen.blit(online_text_on, online_status_pos)
                else:
                    screen.blit(online_text_off, online_status_pos)
                    
                screen.blit(copyright_text, copyright_pos)
                
                # update
                pygame.display.update()

                for event in pygame.event.get():
                    if event.type == KEYDOWN:
                        if event.key == pygame.K_SPACE and state == States.IDLE:
                            micro_img = micro_img_white
                            state = States.LISTENING
                            log("space")
                            cap.release()
                            cap = cv2.VideoCapture(listen_video_file)

                            listening_thread = VoiceRecordingThread()
                            listening_thread.start()
                            log("listening thread is started")

                        elif event.key == pygame.K_q:
                            # exit
                            #sys.exit(0)
                            raise StopExecution
                        elif event.key == pygame.K_i:
                            online_status = not online_status
                        elif event.key == pygame.K_b:
                            # break and go to idle cycle
                            if stop_event:
                                stop_event.set()
                            speaking_thread = None
                            listening_thread = None
                            state = States.IDLE
                            cap.release()
                            cap = cv2.VideoCapture(idle_video_file)
                            #sys.exit(0)
                    elif event.type == KEYUP:
                        if event.key == pygame.K_SPACE:
                            micro_img = micro_img_black
                            if listening_thread:
                                listening_thread.recognizer.energy_threshold = 10000
                    elif event.type == pygame.QUIT:
                        raise StopExecution
                        #exit()
                        #pygame.quit()
                        #cv2.destroyAllWindows()

                cv2.waitKey(25)
                
            else:
                if state == States.IDLE:
                    # random play hello
                    if state == States.IDLE:
                        if random() < 0.5 and not hello_played:
                            hello_played = True
                            rand_video_id = randint(0, len(special_videos)-1)
                            cap = cv2.VideoCapture(special_videos[rand_video_id][0])
                            mixer.init()
                            mixer.music.load(special_videos[rand_video_id][1])
                            mixer.music.play()
                            
                           # mixer.music.unload()
                        else:
                            hello_played = False
                            cap = cv2.VideoCapture(idle_video_file)
                    
                elif state == States.LISTENING:
                    cap = cv2.VideoCapture(listen_video_file)
                else:
                    cap = cv2.VideoCapture(speaking_video_file)
                    
    except (KeyboardInterrupt, SystemExit, StopExecution):
        log("stopped")
        pygame.quit()
        cv2.destroyAllWindows()

        
class StopExecution(Exception):
    def _render_traceback_(self):
        pass



def shift_pitch(filename, higher_pitch_shift, lower_pitch_shift, load_original=False):
    audio_wavelength, sampling_rate = librosa.load(filename)
    if load_original:
        IPython.display.display(Audio(data=audio_wavelength, rate=sampling_rate, autoplay=False))
    if higher_pitch_shift > 0:
        higher_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=higher_pitch_shift)
        IPython.display.display(Audio(data=higher_pitch, rate=sampling_rate, autoplay=False))
    if lower_pitch_shift < 0:
        lower_pitch = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=lower_pitch_shift)
        IPython.display.display(Audio(data=lower_pitch, rate=sampling_rate, autoplay=False))
        
def save_shifted_pitch(output_voice_file, output_voice_file_changed, pitch_shift, load_original=False, load_altered=False):
    audio_wavelength, sampling_rate = librosa.load(output_voice_file)
    altered = librosa.effects.pitch_shift(audio_wavelength, sr=sampling_rate, n_steps=pitch_shift)
    sf.write(output_voice_file_changed, altered, sampling_rate)

In [17]:
# main code
if __name__ == '__main__':
    unicornVideo()

space
listening thread is started
speak


Exception in thread Thread-273:
Traceback (most recent call last):
  File "C:\Users\vadim\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-636fe6c549a1>", line 42, in run
  File "C:\Users\vadim\anaconda3\lib\site-packages\speech_recognition\__init__.py", line 728, in recognize_google
    if not isinstance(actual_result, dict) or len(actual_result.get("alternative", [])) == 0: raise UnknownValueError()
speech_recognition.exceptions.UnknownValueError




space
listening thread is started
speak
User input: dares best best
scores: 0.19767262 40
online, sending request
Model response: dares
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak
User input: who are you
scores: 0.5535895 1
use predefined question:  What is your name?
Answers/0101_response_unicorn.mp3
Answers/0101_response_unicorn.mp4
audio end
space
listening thread is started
speak
User input: asbestos
scores: 0.35481828 4
online, sending request
Model response: Asbestos is a naturally occurring mineral fiber that has been used in a variety of building materials since the 19th century. However, due to its carcinogenic properties, it has been banned in many countries.
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak


Exception in thread Thread-289:
Traceback (most recent call last):
  File "C:\Users\vadim\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-636fe6c549a1>", line 42, in run
  File "C:\Users\vadim\anaconda3\lib\site-packages\speech_recognition\__init__.py", line 728, in recognize_google
    if not isinstance(actual_result, dict) or len(actual_result.get("alternative", [])) == 0: raise UnknownValueError()
speech_recognition.exceptions.UnknownValueError




space
listening thread is started
speak
User input: how will I win the lottery
scores: 0.29165986 37
online, sending request
Model response: I'm afraid I can't answer that question. I'm an AI assistant, not a fortune teller.
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak
User input: when you
scores: 0.37772584 3
online, sending request
Model response: say it can summarize long emails, does it just give a summary of the main points or does it also include the key details? MailboxIQ can summarize long emails into a few sentences that capture the main points and key details. This can save time for users and help them to quickly get the gist of an email without having to read the entire thing.
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak
User input: can you can you tell me what is the time now
scores: 0.29133058 16
online, sending request
Model response: The time is currently 10:15 AM.
response_chan

...

User input: lease explain to a 5-year old why do you Earth is a sphere
scores: 0.18823275 4
online, sending request
Model response: The Earth is a sphere because it is made up of many different materials that are all pulled together by gravity. The heavier materials, like the core and mantle, are closer to the center of the Earth, while the lighter materials, like the crust, are closer to the surface. This creates a round shape, just like a ball.
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak
User input: so hello what's your name
scores: 0.72282064 1
use predefined question:  What is your name?
Answers/0101_response_unicorn.mp3
Answers/0101_response_unicorn.mp4
audio end
space
listening thread is started
speak
User input: so what's the stock price at this point in time
scores: 0.26737908 31
online, sending request
Model response: I'm not able to answer questions about the stock price.  I can provide information about our application, team, an

Exception in thread Thread-349:
Traceback (most recent call last):
  File "C:\Users\vadim\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-636fe6c549a1>", line 60, in run
  File "<ipython-input-15-636fe6c549a1>", line 92, in voice_prep
  File "C:\Users\vadim\anaconda3\lib\site-packages\gtts\tts.py", line 128, in __init__
    assert text, "No text to speak"
AssertionError: No text to speak


Model response: 


space
listening thread is started
speak
User input: go to bank
scores: 0.46167853 2
online, sending request
Model response: MailboxIQ is an AI-powered application that helps you manage your emails more efficiently. It can summarize long emails, analyze the sentiment of your emails, and translate them to other languages.
response_changed.mp3
assets/speaking.mp4
audio end
space
listening thread is started
speak


Exception in thread Thread-359:
Traceback (most recent call last):
  File "C:\Users\vadim\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-636fe6c549a1>", line 42, in run
  File "C:\Users\vadim\anaconda3\lib\site-packages\speech_recognition\__init__.py", line 728, in recognize_google
    if not isinstance(actual_result, dict) or len(actual_result.get("alternative", [])) == 0: raise UnknownValueError()
speech_recognition.exceptions.UnknownValueError




space
listening thread is started
speak
User input: how are you
scores: 0.8818229 18
use predefined question:  How are you?
Answers/0110_response_unicorn.mp3
Answers/0110_response_unicorn.mp4
audio end
space
listening thread is started
speak
User input: how do I choose what to do with later
scores: 0.30270672 3
online, sending request
Model response: emails MailboxIQ can help you choose what to do with later emails by using its priority sorting feature. This feature sorts emails into three categories: important, urgent, and other. You can then choose to deal with the important emails first, followed by the urgent emails, and finally the other emails.
response_changed.mp3
assets/speaking.mp4
audio end
stopped


In [17]:
#voice_prep("I'm sorry, I got disconnected from the Internet and I can't provide an answer to that question. Please feel free to reach out to the person at our exhibition stand for assistance. They will be happy to help with all your queries and provide you with more information about our products and services. Thank you for your understanding.")

In [18]:
#PlaySoundThread(Event()).run()

audio end


In [20]:
#sr.Microphone.list_microphone_names()

['Microsoft Sound Mapper - Input',
 'Mikrofon (TONOR TC30 Audio Devi',
 'Mikrofonarray (IntelÂ® Smart Sou',
 'KopfhÃ¶rermikrofon (JBL Clip 2 H',
 'Microsoft Sound Mapper - Output',
 'Lautsprecher (JBL Clip 2 Stereo',
 'KopfhÃ¶rer (JBL Clip 2 Hands-Fre',
 'Lautsprecher (Realtek(R) Audio)',
 'PrimÃ¤rer Soundaufnahmetreiber',
 'Mikrofon (TONOR TC30 Audio Device)',
 'Mikrofonarray (IntelÂ® Smart Sound Technologie fÃ¼r digitale Mikrofone)',
 'KopfhÃ¶rermikrofon (JBL Clip 2 Hands-Free AG Audio)',
 'PrimÃ¤rer Soundtreiber',
 'Lautsprecher (JBL Clip 2 Stereo)',
 'KopfhÃ¶rer (JBL Clip 2 Hands-Free AG Audio)',
 'Lautsprecher (Realtek(R) Audio)',
 'Lautsprecher (JBL Clip 2 Stereo)',
 'KopfhÃ¶rer (JBL Clip 2 Hands-Free AG Audio)',
 'Lautsprecher (Realtek(R) Audio)',
 'Mikrofon (TONOR TC30 Audio Device)',
 'Mikrofonarray (IntelÂ® Smart Sound Technologie fÃ¼r digitale Mikrofone)',
 'KopfhÃ¶rermikrofon (JBL Clip 2 Hands-Free AG Audio)',
 'KopfhÃ¶rer ()',
 'Lautsprecher ()',
 'Mikrofonarray 1 ()',
 'M

## TODO:
* log